In [24]:
import os
os.chdir("..")

In [ ]:
import django
# In case that we need it later
#from django.conf import settings
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'DevEstate.settings')
# This is for async, in case we are going to see it later (maybe)
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [30]:
# WORKING API CALL

#Import Necessities: SSL is a Temporary fix for a handshake bug, will fix later
import http.client
import ssl
import json
ssl._create_default_https_context = ssl._create_unverified_context

# Returns a string of JSON Information regarding every property in New York City
def fetch_all_data():
   conn = http.client.HTTPSConnection("zillow-com1.p.rapidapi.com")
   headers = {
        'X-RapidAPI-Key': "0b6fb86389msh7cc646b4043d48bp1ff2e0jsn0e456ee9ca9b",
        'X-RapidAPI-Host': "zillow-com1.p.rapidapi.com"
   }
   conn.request("GET", "/searchByUrl?url=https://www.zillow.com/new-york-ny/?searchQueryState=%7B%22pagination%22%3A%7B%7D%2C%22isMapVisible%22%3Atrue%2C%22mapBounds%22%3A%7B%22west%22%3A-74.15820883203124%2C%22east%22%3A-73.66382406640624%2C%22south%22%3A40.57279370530749%2C%22north%22%3A40.93585324525284%7D%2C%22regionSelection%22%3A%5B%7B%22regionId%22%3A6181%2C%22regionType%22%3A6%7D%5D%2C%22filterState%22%3A%7B%22sort%22%3A%7B%22value%22%3A%22globalrelevanceex%22%7D%2C%22ah%22%3A%7B%22value%22%3Atrue%7D%7D%2C%22isListVisible%22%3Atrue%2C%22mapZoom%22%3A11%7D", headers=headers)
   res = conn.getresponse()
   data = res.read()
   return(data.decode("utf-8"))

#Converting that string into a Dictionary
listingsInfo = json.loads(fetch_all_data())
print(listingsInfo["props"][1]["zpid"])

#Returns a variety of information on a property based on its unique ZPID (Zillow ID)
def fetch_individual_data(ZPID):
    conn = http.client.HTTPSConnection("zillow-com1.p.rapidapi.com")
    headers = {
    'X-RapidAPI-Key': "0b6fb86389msh7cc646b4043d48bp1ff2e0jsn0e456ee9ca9b",
    'X-RapidAPI-Host': "zillow-com1.p.rapidapi.com"
    }
    conn.request("GET", "/property?zpid="+str(ZPID), headers=headers)
    res = conn.getresponse()
    data = res.read()
    return(data.decode("utf-8"))

#Parses through a Dictionary of Listings in order to obtain each one's ZPID; Then, fetches the data associated with each one
def parseListings(listingData):
    for i in range(len(listingData)):
        print(fetch_individual_data(listingData["props"][i]["zpid"]))
        #fill out card info from here @ adeeb
parseListings(listingsInfo)


2056637647
{"listingProvider":{"enhancedVideoURL":null,"showNoContactInfoMessage":false,"postingGroupName":"Compass","isZRMSourceText":null,"showLogos":null,"logos":{},"sourceText":"Source: StreetEasy","title":"Listing Provided by","disclaimerText":null,"postingWebsiteURL":"https:\/\/www.compass.com\/app\/listing\/431-east-32nd-street-brooklyn-ny-11226\/1393041771899514889","agentLicenseNumber":null,"postingWebsiteLinkText":"Compass","enhancedDescriptionText":null,"agentName":"Louis Belisario - Licensed Associate Real Estate Broker"},"zpid":30662254,"buildingPermits":null,"propertyTaxRate":0.72,"contact_recipients":[{"agent_reason":1,"zpro":null,"recent_sales":2,"review_count":148,"display_name":"Keystone Realty Of Greater New York","zuid":"X1-ZUysbco5sfy6mh_z47i","rating_average":5,"badge_type":"Premier Agent","phone":{"prefix":"788","areacode":"516","number":"6304"},"image_url":"https:\/\/photos.zillowstatic.com\/h_n\/ISmupqcn3q99ky0000000000.jpg"}],"zipcode":"11226","openHouseSchedu

In [ ]:
def fetch_individual_data(ZPID):

    conn = http.client.HTTPSConnection("zillow-com1.p.rapidapi.com")

    headers = {
    'X-RapidAPI-Key': "0b6fb86389msh7cc646b4043d48bp1ff2e0jsn0e456ee9ca9b",
    'X-RapidAPI-Host': "zillow-com1.p.rapidapi.com"
    }

    conn.request("GET", "/property?zpid="+ZPID, headers=headers)

    res = conn.getresponse()
    data = res.read()
    return(data.decode("utf-8"))

In [4]:
def parseListings(listingData):
    for i in listingData.len():
        print(fetch_individual_data(listingData["props"][i]["zpid"]))
        #fill out card info from here @ adeeb
parseListings(listingsInfo)

NameError: name 'listingsInfo' is not defined